In [1]:
from pyomo.environ import *
import numpy as np
import pandas as pd
from read import *

In [2]:
def GE_0_Bound_V1(model, i, j):
    return (0, None)
    
def GE_0_Bound_V2(model, i):
    return (0, None)



In [3]:
data = read_model_2("Clients/DER.xlsx", "Clients/Export.csv", "Clients/Import.csv", "Clients/Wholesale prices.csv", "Clients/Inflexible load.csv", "Clients/PV.csv")
inputs = OrderedDict()
inputs["start_client"]   = 0
inputs["end_client"]     = 0
inputs["number_clients"] = 1 # max : 92
inputs["FCAS"]           = True
outputs = initialization(inputs, data)

In [4]:
m = ConcreteModel()
    
#! indices
# time intervals
lenT    = len(data["T"])
m.T     = Set(initialize = [t for t in data["T"]])
m.T_SOC = Set(initialize = [t for t in range(lenT + 1)])
# FCAS markets
m.W     = Set(initialize = [w for w in data["W"]])

#! parameters    
# inflexible load profile (kW)
m.P_il = Param(m.T, initialize=data["load"].iloc[:, 0])
# PV generation profile (kW)
m.MPV  = Param(m.T, initialize=data["PV"].iloc[:, 0])
# TODO maximum charging power of the BESS (kW) 
MP_C = data["power"][0]
# TODO maximum discharging power of the BESS (kW) 
MP_D = data["power"][0]
# TODO minimum, maximum state-of-charge of the BESS (kW)
SOC_min = data["Min_SOC"]
SOC_max = data["Max_SOC"][0]
# TODO Tariff: buy price ($/kWh)
m.λ_TB = Param(m.T,      initialize=data["tariff_buy"][str(0)])
# TODO Tariff: sell price ($/kWh)
λ_TS = data["tariff_sell"][str(0)]
# TODO wholesale price ($/kWh)
m.λ_E  = Param(m.T,      initialize=data["energy_price"])
# raise FCAS price ($/kW) 
def init_Param_RFCAS(model, t, w):
    return data["R_FCAS_price"][t, w]
m.λ_R = Param(m.T, m.W, initialize=init_Param_RFCAS)
# lower FCAS price ($/kW) 
def init_Param_LFCAS(model, t, w):
    return data["L_FCAS_price"][t, w]
m.λ_L = Param(m.T, m.W, initialize=init_Param_LFCAS)
# length of the time interval t (h)
Δt  = data["Δt"]
# TODO efficiency of the BESS
η   = data["eff"][0]
# TODO upper bound of energy bids (kW)
m.E_U = Param(m.T, initialize=(data["load"].iloc[:, 0] + MP_C))
# TODO lower bound of energy bids (kW)
m.E_L = Param(m.T, initialize=(data["load"].iloc[:, 0] - data["PV"].iloc[:, 0] - MP_D))

In [5]:
#! variables
# energy bids (kW)
m.E    = Var(m.T,      within=Reals)
# Lower FCAS bids (kW) 
m.L    = Var(m.T, m.W, within=Reals, bounds=GE_0_Bound_V1)
# Raise FCAS bids (kW) 
m.R    = Var(m.T, m.W, within=Reals, bounds=GE_0_Bound_V1)
# Lower capacity provided by BESS (kW)
m.L_c  = Var(m.T,      within=Reals, bounds=GE_0_Bound_V2)
m.L_d  = Var(m.T,      within=Reals, bounds=GE_0_Bound_V2)
# Raise capacity provided by BESS (kW)
m.R_c  = Var(m.T,      within=Reals, bounds=GE_0_Bound_V2)
m.R_d  = Var(m.T,      within=Reals, bounds=GE_0_Bound_V2)
# Raise/lower capacity provided by PV (kW)
m.L_pv = Var(m.T,      within=Reals, bounds=GE_0_Bound_V2)
m.R_pv = Var(m.T,      within=Reals, bounds=GE_0_Bound_V2)
# charging/discharging power of the BESS (kW)
m.P_c  = Var(m.T,      within=Reals, bounds=GE_0_Bound_V2)
m.P_d  = Var(m.T,      within=Reals, bounds=GE_0_Bound_V2)
# PV generation (kW)
m.PV   = Var(m.T,      within=Reals)
# state-of-charge (kWh)
m.SOC  = Var(m.T_SOC,  within=Reals)
# charging or discharging (0 or 1)
m.τ    = Var(m.T,      within=Binary)
# TODO positive or negative m.E (0 or 1) 
m.α    = Var(m.T,      within=Binary)
# TODO total cost
def EB_Bound(model, i):
    return (0, m.E_U[i])
def ES_Bound(model, i):
    return (min(0, m.E_L[i]), 0)
m.E_b  = Var(m.T,      within=Reals, bounds=EB_Bound)
m.E_s  = Var(m.T,      within=Reals, bounds=ES_Bound)

In [6]:
#! objective function
def obj_rule(m):
    return (sum(λ_TS * m.E_s[t] + m.λ_TB[t] * m.E_b[t] for t in data["T"])) * Δt - (summation(m.λ_R, m.R)) * Δt - (summation(m.λ_L, m.L)) * Δt
m.obj = Objective(rule=obj_rule, sense=minimize)

In [7]:
#! constraints
# Constraint (18) defines the consumption or generation of the prosumer
def Constraint_18(m, t):
    return m.E[t] == m.P_c[t] - m.P_d[t] + m.P_il[t] - m.PV[t]
m.constrs18 = Constraint(m.T, rule = Constraint_18)

# Constraints (19) and (20) define the raise and lower capacities traded in each FCAS market
def Constraint_19(m, t, w):
    return m.R[t, w] <= m.R_c[t] + m.R_d[t] + m.R_pv[t]
m.constrs19 = Constraint(m.T, m.W, rule = Constraint_19)

def Constraint_20(m, t, w):
    return m.L[t, w] <= m.L_c[t] + m.L_d[t] + m.L_pv[t]
m.constrs20 = Constraint(m.T, m.W, rule = Constraint_20)

# Constraints (24) and (25) set the ranges of the discharging power and charging power
def Constraint_24(m, t):
    return m.P_d[t] <= (1 - m.τ[t]) * MP_D
m.constrs24 = Constraint(m.T, rule = Constraint_24)

def Constraint_25(m, t):
    return m.P_c[t] <= m.τ[t] * MP_C
m.constrs25 = Constraint(m.T, rule = Constraint_25)

# Constraints (26) and (27) set the state-of-charge SOC[t+1] of the BESS within its limits
def Constraint_26(m, t):
    return m.SOC[t + 1] == m.SOC[t] + (m.P_c[t] * η - m.P_d[t] * (1 / η)) * Δt
m.constrs26 = Constraint(m.T, rule = Constraint_26)

def Constraint_27(m, t):
    return (SOC_min, m.SOC[t + 1], SOC_max)
m.constrs27 = Constraint(m.T, rule = Constraint_27)

# Constraints (28) and (29) bound the raise capacity
def Constraint_28(m, t):
    return m.R_d[t] <= MP_D - m.P_d[t]
m.constrs28 = Constraint(m.T, rule = Constraint_28)

def Constraint_29(m, t):
    return m.R_c[t] <= m.P_c[t]
m.constrs29 = Constraint(m.T, rule = Constraint_29)

# Constraints (30) and (31) bound the lower capacity
def Constraint_30(m, t):
    return m.L_c[t] <= MP_C - m.P_c[t]
m.constrs30 = Constraint(m.T, rule = Constraint_30)

def Constraint_31(m, t):
    return m.L_d[t] <= m.P_d[t]
m.constrs31 = Constraint(m.T, rule = Constraint_31)

# Constraints (32) and (33) ensure that the BESS only provides lower and raise capacities, when SOC[t+1] is within its technical limits.
def Constraint_32(m, t):
    return (m.L_c[t] * η + m.L_d[t] * (1 / η)) * Δt <= SOC_max - m.SOC[t + 1]
m.constrs32 = Constraint(m.T, rule = Constraint_32)

def Constraint_33(m, t):
    return (m.R_c[t] * η + m.R_d[t] * (1 / η)) * Δt <= m.SOC[t + 1] - SOC_min  
m.constrs33 = Constraint(m.T, rule = Constraint_33)

# Constraint (34) sets the range of pv generation based on the forecasted solar power
def Constraint_34(m, t):
    return (0, m.PV[t], m.MPV[t])
m.constrs34 = Constraint(m.T, rule = Constraint_34)

# Constraint (35) and (36) define the raise and lower capacities of the PV
def Constraint_35(m, t):
    return m.R_pv[t] <= m.MPV[t] - m.PV[t]
m.constrs35 = Constraint(m.T, rule = Constraint_35)

def Constraint_36(m, t):
    return m.L_pv[t] <= m.PV[t]
m.constrs36 = Constraint(m.T, rule = Constraint_36)

# SOC Constraint
m.socc = Constraint(expr=m.SOC[0] == 0)

In [8]:
# TODO positive or negative Constraint
def Constraint_Pos_1(m, t):
    return m.E[t] <= m.E_U[t] * m.α[t]
m.constrsPos_1 = Constraint(m.T, rule = Constraint_Pos_1)

def Constraint_Pos_2(m, t):
    return m.E[t] >= m.E_L[t] * (1 - m.α[t])
m.constrsPos_2 = Constraint(m.T, rule = Constraint_Pos_2)

In [9]:
def Constraint_PALL(m, t):
    return m.E[t] == m.E_b[t] + m.E_s[t]
m.constrsAll = Constraint(m.T, rule = Constraint_PALL)

def Constraint_EBuy(m, t):
    return m.E_b[t] <= m.E_U[t] * m.α[t]
m.constrsEBuy = Constraint(m.T, rule = Constraint_EBuy)

def Constraint_ESell(m, t):
    return m.E_s[t] >= m.E_L[t] * (1 - m.α[t])
m.constrsESell = Constraint(m.T, rule = Constraint_ESell)

In [10]:
opt        = SolverFactory('cplex')  # opt = SolverFactory('glpk')
solution   = opt.solve(m, tee=False)
print(solution.solver.termination_condition) 
print(solution.solver.termination_message) 
print(solution.solver.status)

optimal
MIP - Integer optimal, tolerance (0.0001/1e-06)\x3a Objective = -2.3702601484e+03
ok


In [ ]:
# # TODO Cost Constraint
# def Constraint_ESell_1(m, t):
#     return m.C[t] - m.E[t] * λ_TS <= (m.λ_TB[t] * m.E_U[t] - λ_TS * m.E_L[t]) * m.α[t]
# m.constrsESell_1 = Constraint(m.T, rule = Constraint_ESell_1)

# def Constraint_ESell_2(m, t):
#     return m.C[t] - m.E[t] * λ_TS >= (λ_TS * m.E_L[t]) * m.α[t]
# m.constrsESell_2 = Constraint(m.T, rule = Constraint_ESell_2)

# def Constraint_EBuy_1(m, t):
#     return m.C[t] - m.E[t] * m.λ_TB[t] <= (m.λ_TB[t] * m.E_U[t]) * (1 - m.α[t])
# m.constrsEBuy_1 = Constraint(m.T, rule = Constraint_EBuy_1)

# def Constraint_EBuy_2(m, t):
#     return m.C[t] - m.E[t] * m.λ_TB[t] >= (λ_TS * m.E_L[t] - m.λ_TB[t] * m.E_U[t]) * (1 - m.α[t])
# m.constrsEBuy_2 = Constraint(m.T, rule = Constraint_EBuy_2)